# Web API: cnyes & 104.com

- [slide](https://docs.google.com/presentation/d/1gn4d5gzXzgyEAIz_AOdM3pmJ0-6iiN7EbTjKqCQlgPo/edit?usp=sharing) for getting json data online
- Finding data url
- Scraping 104.com job info
- Understanding how to send request and get back response
- Send request with Cookie, payload, Referer...


## Send web requests

* Document: http://docs.python-requests.org/en/master/ 
* Quickstart http://docs.python-requests.org/en/master/user/quickstart/

瀏覽器在瀏覽一個頁面時，實際上所做的動作是對對對方伺服器發出一個HTTP要求（request），對方伺服器會根據這個要求合不合法來決定要給對方什麼樣的回應（response）。這些回應主要有如[HTTP Status Code](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)上所有的情形。例如對方伺服器如果發現要求的內容是對的，但權限是有問題的，就會回應一個「403 Forbidden」的狀態碼（Status Code）。而如果能夠順利拿回資料的話，便會回應「200 OK」並把所要求的資料在回應中傳回來。

HTTP要求有兩類，分別是`GET()`和`POST()`。`POST()`通常是要填寫表單資訊的時候所送出的要求，例如需要指明要哪一區、哪一種類型的房子、或哪一種類型的車票等。而一般新聞網站或社群網站多會用`GET()`取回資料。

In [1]:
import requests
import json
response = requests.get('https://tcgbusfs.blob.core.windows.net/blobyoubike/YouBikeTP.gz')
response.json()
type(response.json())

dict

### Practice: Find out and traverse data behind urls

逐一測試過每一個link是否都讀得到資料，以獲得對這些資料概略了解。
* (hint) Using `requests.get(url, timeout=(x, y))` to set the limitation of waiting time
* `timeout=(x, y)`: Max x seconds to connect to server and max y seconds to wait on response
* https://data.moi.gov.tw/moiod/System/Principle.aspx?Sample=2

#### Canyes

In [39]:
url_cnyes = "https://news.cnyes.com/api/v3/news/category/headline?startAt=1588262400&endAt=1589212799&limit=30"
res = requests.get(url_cnyes).json()
print(type(res))
print(res.keys())

<class 'dict'>
dict_keys(['items', 'message', 'statusCode'])


#### PChome 24h product search

In [35]:
url_pchome = "https://ecshweb.pchome.com.tw/search/v3.3/all/results?q=iphone&page=1&sort=sale/dc"
res = requests.get(url_pchome).json()
print(type(res))
print(res.keys())

<class 'dict'>
dict_keys(['QTime', 'totalRows', 'totalPage', 'range', 'cateName', 'q', 'subq', 'token', 'isMust', 'prods'])


#### Dcard

Dcard API最近又做了改版，用以下的Code應該沒辦法順利撈回，但你也可以測試看看，可以看到什麼樣的錯誤訊息。

In [46]:
url_dcard = "https://www.dcard.tw/service/api/v2/forums/relationship/posts?limit=50"
user_agent_dcard = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.34"
headers = {'User-Agent': user_agent_dcard}
res = requests.get(url_dcard, headers = headers)
print(res)

### (Option) Write a function to load json data

In [49]:
def get_web_json(url, headers=""):
    response = requests.get(url, timeout=(3, 5), headers=headers)
    print("Response Code:", response.status_code)
    if not response.ok:
        return None
    data = response.json()
    return data

# get_web_json(url_pchome)
# get_web_json(url_cnyes)

## Scraping 104.com
* Slide: https://docs.google.com/presentation/d/e/2PACX-1vRW84XoB5sFRT1Eg-GrK4smX23qoNkFffz_h8oRU4AIvJAgrrxBn8059_0UeHv_pFBks_Z37vNbLGai/pub?start=false&loop=false&delayms=3000


Address

```
https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=2&mode=s&jobsource=2018indexpoc
```

### Step01. Get the 1st page (but fails)

先嘗試用上面的方法獲取104.com的搜尋結果的第一頁資料網址，結果應該會傳回來一個空的HTML無法傳回資料。這是因為通常服務提供方會嘗試要求提出拜訪要求的瀏覽器需要提供一些簡單的驗證機制，例如是用什麼瀏覽器連上去的（稱為User-agent，DCard會要求這樣的資訊）、是從哪一個頁面跳過去的（稱為Referer，104.com就有這樣的要求）。

撰寫爬蟲時必須「模仿」瀏覽器的機制，如果對方希望瀏覽器提供這樣的資訊，那撰寫爬蟲時也就要提供這樣的資訊。


In [51]:
url_104 = 'https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=2&mode=s&jobsource=2018indexpoc'
response = requests.get(url_104)
print(response.status_code)

200


In [52]:
print(response.text)

<html xmlns="http://www.w3.org/1999/xhtml"><head><META HTTP-EQUIV="CONTENT-TYPE" CONTENT="TEXT/HTML; CHARSET=utf-8"/><title></title></head>
<body>
<SCRIPT LANGUAGE="JavaScript">
window.location="https://www.104.com.tw/jobs/main/syserr?eid=6129352555597738053";
</script>
</body>
</html>


In [53]:
print(response.headers)

{'Cache-Control': 'no-cache', 'Connection': 'close', 'Pragma': 'no-cache', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '292'}


#### Trick. Write html to file

當明明知道他是一個json，卻一直無法順利用`.json()`將其拆且為`list` or `dict`的型態時，最有可能的問題是，該網址設了一些檢核機制讓我們不能那麼粗暴草率地拿到資料，導致它傳回來給我們的是一個用html編寫成的錯誤訊息，例如stuatus code: 403。這時候我要怎麼知道發生錯誤？一種方式是把status code印出來看看；另一種方式是，他可能也是傳給你status code:200，但實際上就是傳回一個告知你不能存取的html。這時候我們可以採取的作法就是把該HTML，也就是回傳的結果寫入到.html檔，然後用瀏覽器開啟看看他究竟傳回來什麼錯誤訊息。

觀察上面`response.text`中的回應顯然是以HTML格式撰寫，而下方程式碼會把回應內容寫到一個HTML檔案裡面去，可以在本機端或CoLab上點開該HTML檔來觀察看看究竟傳回來什麼東西。

In [10]:
with open('temp_output.html', 'w') as fout:
    fout.write(response.text)

# webbrowser cannot work, why?
import webbrowser
webbrowser.open_new_tab('temp_output.html')

True

#### Practice. Observe youbike data headers

Youtube data所回傳的是一個JSON檔，而上述104.com目前回傳的資料內容是個HTML檔，觀察看看，youbike回應標題是什麼（response headers）？如果要很會寫爬蟲，一定要會觀察header，而這只是最簡單的一種情形。


In [55]:
import requests
import json
response = requests.get('https://tcgbusfs.blob.core.windows.net/blobyoubike/YouBikeTP.gz')

print(response)
print(response.status_code)
print(type(response)) # <class 'requests.models.Response'>
print(type(response.text)) # <class 'str'>

<Response [200]>
200
<class 'requests.models.Response'>
<class 'str'>


In [56]:
print(response.headers)
import pandas as pd
print(pd.DataFrame.from_dict(response.headers, orient='index'))

{'Content-Length': '30318', 'Content-Type': 'application/octet-stream', 'Content-Encoding': 'gzip', 'Content-MD5': 'yKWKnRwXkraz2fHOw4yhiQ==', 'Last-Modified': 'Mon, 10 Oct 2022 05:17:54 GMT', 'ETag': '0x8DAAA7EC93CFD70', 'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0', 'x-ms-request-id': 'ef024e13-b01e-00cc-2d67-dc9e66000000', 'x-ms-version': '2009-09-19', 'x-ms-lease-status': 'unlocked', 'x-ms-blob-type': 'BlockBlob', 'Access-Control-Allow-Origin': '*', 'Date': 'Mon, 10 Oct 2022 05:18:40 GMT'}
                                                                        0
Content-Length                                                      30318
Content-Type                                     application/octet-stream
Content-Encoding                                                     gzip
Content-MD5                                      yKWKnRwXkraz2fHOw4yhiQ==
Last-Modified                               Mon, 10 Oct 2022 05:17:54 GMT
ETag                                          

### Step02. Add Referer to get 104 data

現在我們來加入一些對方伺服器可能會要求我們發出request的時候提供的東西，最常見的有以下的資訊。但，實際上我們得一個一個測試看看，才會知道對方要求什麼。通常會從User-agent測起，再來Referer、再來Cookie。但有經驗的人多半一看就猜得到。

* User-Agent: 你用什麼瀏覽器或系統
* Referer: 你從哪個頁面點選、跳轉過來
* Cookies: 經過與伺服器建立連結後，他給了你什麼資訊好讓你持續可以待在這個頁面。

下方程式碼在送出`GET()`要求時也一併送出了Referer的資訊`'https://www.104.com.tw/'`，意義相當於偽裝成先打開104.com的首頁後，從首頁進入這個查詢連結。理論上如果沒進入首頁，就直接進入查詢連結，不是非常可疑嗎？所以，有些伺服器會偵測，這個要求是從哪一個頁面網址點選後轉過來的，

In [57]:
url_104 = 'https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=2&mode=s&jobsource=2018indexpoc'

headers = {'referer': 'https://www.104.com.tw/'}

raw = requests.get(url_104, headers=headers).json()
print(type(raw))

<class 'dict'>


### Step03. Traverse data to get the data block

觀察一下資料的概況，並把他轉為pandas來觀察它。

In [58]:
print(raw.keys())
print(type(raw['data']))
print(raw['data'].keys())
print(type(raw['data']['list']))
print(type(raw['data']['list'][0]))
df = pd.DataFrame(raw['data']['list'])
df

dict_keys(['status', 'action', 'data', 'statusMsg', 'errorMsg'])
<class 'dict'>
dict_keys(['query', 'filterDesc', 'queryDesc', 'list', 'count', 'pageNo', 'totalPage', 'totalCount'])
<class 'list'>
<class 'dict'>


,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,tags,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,major
0,0,10589685,Data Scientist 資料科學家_數據分析師 | CBU_21_13,<em class='b-txt--highlight'>Data</em> <em cla...,1,1,6001002020,新北市三重區,三和路四段111之32號7樓,分群、推薦、迴歸預測等演算法理論基礎，並有實作經驗\r\n2. 熟悉程式與分析應用，如 Py...,...,[員工300人],距捷運三和國中站110公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,Data Scientist 資料科學家_數據分析師 | CBU_21_13,伊雲谷數位科技股份有限公司,121.4856118,25.0774577,0,[]
1,0,10856947,數據科學家 Data Scientist,<em class='b-txt--highlight'>數據科學家</em> <em cl...,1,1,6001001007,台北市信義區,松仁路100號10樓,Isobar作為數位轉型與數位創新的領導品牌，領先市場佈局打造專業的CR-Marketing...,...,[員工120人],距捷運象山站290公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,數據科學家 Data Scientist,安索帕股份有限公司,121.5678798,25.0344547,0,"[統計學相關, 一般商業學類, 資訊管理相關]"
2,2,12988717,16A-T-0015 Data Scientist 資料科學家,16A-T-0015 <em class='b-txt--highlight'>Data</...,1,1,6001016007,高雄市前鎮區,中六路一號,【工作內容】\n1. 跨部門合作，透過數據挖掘可能的應用。\n2. 統計分析、機器/深度學習...,...,"[上市上櫃, 員工19000人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,16A-T-0015 Data Scientist 資料科學家,瑞儀光電股份有限公司,120.2906511,22.5994751,0,"[應用數學相關, 統計學相關]"
3,0,12844505,Data Scientist資料科學家(遠端工作)-T05P,<em class='b-txt--highlight'>Data</em> <em cla...,1,1,6001001003,台北市中山區,,"•\tDesign, develop, and launch extremely effic...",...,"[外商公司, 員工200人, 遠端工作]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,Data Scientist資料科學家(遠端工作)-T05P,Tata Consultancy Services Limited Taiwan Branc...,121.5427093,25.0792018,2,"[資訊工程相關, 數理統計相關, 資訊管理相關]"
4,2,12909698,Python Data Scientist,Python <em class='b-txt--highlight'>Data</em> ...,1,1,6001016005,高雄市鼓山區,中華一路828號13樓,\n\n職務需求：\n\nPython [[[Data]]] [[[Scientist]]]...,...,[員工45人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,Python Data Scientist,BigGo_樂方股份有限公司,120.2921244,22.6604715,0,[]
5,0,12348710,【AI Team】資料科學家 Data Scientist,【AI Team】<em class='b-txt--highlight'>資料科學家</e...,1,1,6001001007,台北市信義區,東興路41號10樓,【Responsibilities】 \n1. Lead the team to drive...,...,"[員工150人, 遠端工作]",距捷運南京三民站470公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,【AI Team】資料科學家 Data Scientist,愛卡拉互動媒體股份有限公司,121.5666564,25.0480733,2,[]
6,0,12504693,Data Scientist - Analytics (Product) / 資料科學家 -...,<em class='b-txt--highlight'>Data</em> <em cla...,1,1,6001001002,台北市大同區,承德路,be based under Product division and will be p...,...,[員工180人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,Data Scientist - Analytics (Product) / 資料科學家 -...,Pinkoi_香港商果翼科技股份有限公司台灣分公司,121.5169834,25.0504544,0,[]
7,0,11774964,Python programmer/data scientist for quantitat...,Python programmer/<em class='b-txt--highlight'...,1,1,6001001010,台北市內湖區,港墘路200號5樓,Python programmer/[[[data]]] [[[scientist]]] f...,...,[員工95人],距捷運港墘站470公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,Python programmer/data scientist for quantitat...,中塑油品股份有限公司,121.5743267,25.0759802,0,"[數理統計相關, 其他數學及電算機科學相關, 財稅金融相關]"
8,2,12689367,Data Scientist / 資料科學家_11060,<em class='b-txt--highlight'>Data</em> <em cla...,1,1,6001001009,台北市北投區,立功街96號,"learning / deep learning papers, and have the...",...,[員工6000人],距捷運關渡站410公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,Data Scientist / 資料科學家_11060,和碩集團_和碩聯合科技股份有限公司,121.4712373,25.124796,0,"[資訊工程相關, 統計學相關]"
9,2,12110712,Data Scientist (士林),<em class='b-txt--highlight'>Data</em> <em cla...,1,1,6001001008,台北市士林區,承德路四段166號,[[[Data]]]-Centric AI.\n\nThe organization re...,...,"[上市上櫃, 員工4000人]",距捷運劍潭站240公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_a_relevance,Data Scientist (士林),Inventec_英業達股份有限公司,121.5230712,25.0857408,0,"[數學及電算機科學學科類, 統計學相關, 電機電子工程相關]"


### Step04. Get next page: get the 2nd, 1st, 3rd, ..., page urls

接下來要從Chrome Development Tools來觀察，下二頁、三頁、四頁的網址為何（例如以下網址）。然後要去觀察這些網址的變化，應該不難觀察在page=1, page=2, page=3的數字上有所變化。通常這種網址的變化都是有規律性的。
```
https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=2&mode=s&jobsource=2018indexpoc

https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=1&mode=s&jobsource=2018indexpoc

https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=3&mode=s&jobsource=2018indexpoc
```

因為知道有這樣的規律性，所以就拆解要發出的網址，把網址的頁碼的獨立出來，然後用for-loop來走過每個頁面。

In [15]:
for page in range(1, 6):
    url = 'https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=' + str(page) + '&mode=s&jobsource=2018indexpoc'
    headers = {'referer': 'https://www.104.com.tw/'}
    raw = requests.get(url, headers=headers).json()
    print(len(raw['data']['list']))

30
20
20
20
20


除了列印出來以外，用`all_data`這個變項儲存資料，並在for-loop中把每一頁的資料用`extend()`附加到`all_data`中。

以下程式碼要注意的是，因為頁面沒有第0頁，但`range()`預設是從0開始，所以此時要特別寫定`range()`要從1開始。

In [16]:
all_data = []
for page in range(1, 6):
    url = 'https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=' + str(page) + '&mode=s&jobsource=2018indexpoc'
    headers = {'referer': 'https://www.104.com.tw/'}
    raw = requests.get(url, headers=headers).json()
    all_data.extend(raw['data']['list'])
    print(len(all_data))

30
50
70
90
110


最後我們把整筆資料轉為pandas比較好觀察。

In [59]:
df = pd.DataFrame(all_data)
print(df.shape)
print(len(set(df.jobNo)))

(1250, 40)
111


### Step05. Detect ending condition

前面的步驟中我們知道要如何一步一步抓取每一頁的資料，最後還有一個問題就是，抓到什麼時候才要停？通常程式設計師也得留一個線索，才知道要如何在網頁上自動化呈現最後一頁（如同你看網頁時所看到的最後一頁的頁碼）。所以我們得去揣測程式設計師的邏輯，看他是怎麼設計最後一頁的停止機制的。

通常最主要有兩種：
1. 直接顯示最後一頁是多少：那就寫一個爬蟲，直接去偵測這個最後一頁是多少，然後把他當成ending condition。如104.com的例子是有總資料筆數的，除以頁數就可以知道總頁數。
2. 不直接顯示最後一頁是多少（e.g. PCHOME），就設一個夠大的迴圈，讓爬蟲抓抓抓抓到當掉，或者抓到資料沒再新增了，我們就偵測如果抓到的資料是零筆，就讓他跳出迴圈。

In [73]:
print(raw.keys())
print(type(raw['data']))
print(raw['data']['pageNo'])
print("Total Page:", raw['data']['totalPage'])
print(raw['data']['count'])
print(raw['data']['totalCount'])

dict_keys(['status', 'action', 'data', 'statusMsg', 'errorMsg'])
<class 'dict'>
2
Total Page: 150
['7927', '7639', '146', '29', '113', '0', '0']
7927


#### Trick. pprint (pretty-print)
https://docs.python.org/3/library/pprint.html. ... The pprint module provides a capability to “pretty-print” arbitrary Python data structures in a form which can be used as input to the interpreter ...

Example
```python
class pprint.PrettyPrinter(indent=1, width=80, depth=None, stream=None, *, compact=False, sort_dicts=True, underscore_numbers=False)¶
```

In [74]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(raw['data']['list'][:2])

[   {   'appearDate': '20221006',
        'appearDateDesc': '10/06',
        'applyCnt': '00006',
        'applyDate': '',
        'applyDesc': '6~10人應徵',
        'coIndustry': '1001001003',
        'coIndustryDesc': '網際網路相關業',
        'custName': '伊雲谷數位科技股份有限公司',
        'custNameRaw': '伊雲谷數位科技股份有限公司',
        'custNo': '130000000028759',
        'descSnippet': '分群、推薦、迴歸預測等演算法理論基礎，並有實作經驗\r\n'
                       '2. 熟悉程式與分析應用，如 Python, R, SPSS, MATLAB, SAS 等\r\n'
                       "3. 熟悉大數據生態系(Big <em class='b-txt--highlight'>Data</em> "
                       'Ecosystem)，如 Hadoop, Spark,',
        'description': '分群、推薦、迴歸預測等演算法理論基礎，並有實作經驗\r\n'
                       '2. 熟悉程式與分析應用，如 Python, R, SPSS, MATLAB, SAS 等\r\n'
                       '3. 熟悉大數據生態系(Big [[[Data]]] Ecosystem)，如 Hadoop, Spark,',
        'isApply': '0',
        'isSave': '0',
        'jobAddrNo': '6001002020',
        'jobAddrNoDesc': '新北市三重區',
        'jobAddress': '三和路四段111之32號7樓',
        'jobName': 'Data 

In [89]:
totalPage = raw['data']['totalPage']


# Uncomment the next line code to scrape from 1 to totalage
# for p in range(1, totalPage):
for p in range(1, 6):
    url = 'https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=data%20scientist&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=' + str(page) + '&mode=s&jobsource=2018indexpoc'
    headers = {'referer': 'https://www.104.com.tw/'}
    raw = requests.get(url, headers=headers).json()
    all_data.extend(raw['data']['list'])
    print(len(all_data))

230
250
270
290
310


### 6. convert to dataframe

In [76]:
df = pd.DataFrame(all_data)
print(df.shape)
df

(1430, 40)


,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,tags,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,major
0,1,12901787,日班倉管員(週休二日免排班),日班倉管員(週休二日免排班),1,1,6001005012,桃園市大園區,,"1.倉儲貨品管理及貨品環境維護 \n2.物流倉儲各項改包進貨及出貨作業,熟悉倉庫生態佳 \n...",...,"[外商公司, 員工650人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,日班倉管員(週休二日免排班),香港商台灣利豐物流有限公司台灣分公司_LF LOGISTICS (TAIWAN) LIMITED,121.193945,25.0492632,0,[]
1,1,11726918,水晶軒專櫃店長(台中區)底薪固定13個月+獎金,水晶軒專櫃店長(台中區)底薪固定13個月+獎金,1,1,6001008004,台中市西區,,百貨公司專櫃儲備幹部及店長職務\r\n1.負責介紹及銷售商品。 \r\n2.提供顧客之接待與...,...,"[外商公司, 員工190人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,水晶軒專櫃店長(台中區)底薪固定13個月+獎金,香港商施華洛世奇有限公司台灣分公司,120.6631289,24.1430604,0,[]
2,1,8422468,(Human Resources) Learning and Development Tea...,(Human Resources) Learning and Development Tea...,1,1,6001002011,新北市新店區,,About HTC\n\nHTC built a vision of the future ...,...,"[上市上櫃, 員工2000人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,(Human Resources) Learning and Development Tea...,宏達電 HTC Corporation_宏達國際電子股份有限公司,121.5394822,24.978282,0,"[心理學相關, 人力資源相關]"
3,1,12388507,[2022科技菁英計畫] 產品測試工程師 Product/Test Engineer [限...,[2022科技菁英計畫] 產品測試工程師 Product/Test Engineer [限...,1,1,6001002015,新北市中和區,興南路一段142號,▋ 關於德州儀器\n德州儀器(TI)是位居類比晶片世界領導地位的半導體設計與製造公司，持續提...,...,"[外商公司, 員工2000人]",距捷運南勢角站320公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,[2022科技菁英計畫] 產品測試工程師 Product/Test Engineer [限...,德州儀器工業股份有限公司,121.5083777,24.9871002,0,[工程學科類]
4,1,12781781,【外商】財務分析副理 Finance Planning &amp; Analysis Ass...,【外商】財務分析副理 Finance Planning &amp; Analysis Ass...,1,1,6001001004,台北市松山區,敦化北路88號13樓,Planning and Budgeting\n- Assist in the prepar...,...,"[外商公司, 員工250人]",距捷運台北小巨蛋站340公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,【外商】財務分析副理 Finance Planning & Analysis Assista...,馬來西亞商白蘭氏三得利股份有限公司台灣分公司,121.5484798,25.0513338,0,"[財稅金融相關, 會計學相關, 企業管理相關]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,2,13009437,Data Specialist,<em class='b-txt--highlight'>Data</em> Specialist,1,1,6001001003,台北市中山區,南京東路三段168號,The [[[Data]]] Specialist is responsible for l...,...,"[外商公司, 員工100人]",距捷運南京復興站230公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_c_relevance,Data Specialist,台灣保樂力加股份有限公司,121.5417771,25.0516218,0,[]
1426,0,12850348,微生物(資深) 研究員_Microbiology Senior(Sr.) Scientist,微生物(資深) 研究員_Microbiology Senior(Sr.) <em class...,1,1,6001002025,新北市五股區,五工六路25號,Position Summary:\nPharmacology Discovery Serv...,...,[員工130人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_c_relevance,微生物(資深) 研究員_Microbiology Senior(Sr.) Scientist,汎球生物科技股份有限公司,121.4485193,25.0627591,0,"[生物學相關, 醫學技術及檢驗相關, 藥學相關]"
1427,2,12823438,資料工程師 Data Engineer,資料工程師 <em class='b-txt--highlight'>Data</em> E...,1,1,6001001005,台北市大安區,近忠孝復興捷運站,與PM/UI/UX、前後端、資料、演算法工程師一同合作，開發與維護產品。\n 2. 負...,...,[員工650人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_c_relevance,資料工程師 Data Engineer,東森新媒體控股股份有限公司,121.5425991,25.0418146,0,[]
1428,0,13029724,Data Engineer 數據工程師,<em class='b-txt--highlight'>Data</em> Enginee...,1,1,6001016015,高雄市路竹區,,do we manage the complexities of our manufact...,...,[員工260人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_c_relevance,Data Engineer 數據工程師,德商默克在台集團_美商慧盛先進科技有限公司台灣分公司,120.2628111,22.8494042,0,"[自然科學學科類, 數學及電算機科學學科類]"


## Dump files for backup

* 因為抓取資料很久很辛苦，所以通常會把它寫到後端的檔案、資料庫，或者雲端的資料庫中。在此，我們選擇把他寫到pandas或者`.json()`比較好讀的json檔。通常不會寫到CSV，因為CSV是以逗點分隔為辨識基礎，但文本中可能也會有逗點，會比較容易出錯。
* 建議若設計了寫入，就立刻讀出來看看，省得不小心寫入錯誤，屆時程式關閉了，資料就取不回來了。
* 還有另外一種檔案是python的pickle檔。pickle是一種暫存檔，也就是我們現在如果有一個變數是pandas dataframe，寫到pickle再讀出來，他還會是一個pandas dataframe，而且資料型態都不會變。這點其實非常好用，因為如果你把一個dataframe轉為json，你要特別注意那些datetime欄位或multiindex的東西被轉成什麼，又是否能夠讀得回來。也就是如果時間物件要寫入到json檔時，理應要先轉成string，然後讀取回來要做分析時，也要再轉回來datetime。但如果你轉為pickle檔，存進去是什麼樣子，拿出來就是什麼樣子。那pickle檔有缺點嗎？有！就和其他的程式例如R或網頁程式無法通用，也無法直接寫到雲端資料庫。


### M1. Dump one variable to json by json library
* https://docs.python.org/3/library/json.html

In [90]:
import json

# Uncomment the following code
# with open('104_list.json', 'w') as outfile:
#     json.dump(all_data, outfile)

### M2. Dump and load json by pandas library

In [78]:
# Uncomment the following code

# with open('104_df.json', 'w') as f:
#     f.write(df.to_json())

In [84]:
# Uncomment the following code
# Read-back

# pd.read_json("104_df.json")

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,tags,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,major
0,1,12901787,日班倉管員(週休二日免排班),日班倉管員(週休二日免排班),1,1,6001005012,桃園市大園區,,"1.倉儲貨品管理及貨品環境維護 \n2.物流倉儲各項改包進貨及出貨作業,熟悉倉庫生態佳 \n...",...,"[外商公司, 員工650人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,日班倉管員(週休二日免排班),香港商台灣利豐物流有限公司台灣分公司_LF LOGISTICS (TAIWAN) LIMITED,121.193945,25.049263,0,[]
1,1,11726918,水晶軒專櫃店長(台中區)底薪固定13個月+獎金,水晶軒專櫃店長(台中區)底薪固定13個月+獎金,1,1,6001008004,台中市西區,,百貨公司專櫃儲備幹部及店長職務\r\n1.負責介紹及銷售商品。 \r\n2.提供顧客之接待與...,...,"[外商公司, 員工190人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,水晶軒專櫃店長(台中區)底薪固定13個月+獎金,香港商施華洛世奇有限公司台灣分公司,120.663129,24.143060,0,[]
2,1,8422468,(Human Resources) Learning and Development Tea...,(Human Resources) Learning and Development Tea...,1,1,6001002011,新北市新店區,,About HTC\n\nHTC built a vision of the future ...,...,"[上市上櫃, 員工2000人]",,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,(Human Resources) Learning and Development Tea...,宏達電 HTC Corporation_宏達國際電子股份有限公司,121.539482,24.978282,0,"[心理學相關, 人力資源相關]"
3,1,12388507,[2022科技菁英計畫] 產品測試工程師 Product/Test Engineer [限...,[2022科技菁英計畫] 產品測試工程師 Product/Test Engineer [限...,1,1,6001002015,新北市中和區,興南路一段142號,▋ 關於德州儀器\n德州儀器(TI)是位居類比晶片世界領導地位的半導體設計與製造公司，持續提...,...,"[外商公司, 員工2000人]",距捷運南勢角站320公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,[2022科技菁英計畫] 產品測試工程師 Product/Test Engineer [限...,德州儀器工業股份有限公司,121.508378,24.987100,0,[工程學科類]
4,1,12781781,【外商】財務分析副理 Finance Planning &amp; Analysis Ass...,【外商】財務分析副理 Finance Planning &amp; Analysis Ass...,1,1,6001001004,台北市松山區,敦化北路88號13樓,Planning and Budgeting\n- Assist in the prepar...,...,"[外商公司, 員工250人]",距捷運台北小巨蛋站340公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,【外商】財務分析副理 Finance Planning & Analysis Assista...,馬來西亞商白蘭氏三得利股份有限公司台灣分公司,121.548480,25.051334,0,"[財稅金融相關, 會計學相關, 企業管理相關]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,2,13009437,Data Specialist,<em class='b-txt--highlight'>Data</em> Specialist,1,1,6001001003,台北市中山區,南京東路三段168號,The [[[Data]]] Specialist is responsible for l...,...,"[外商公司, 員工100人]",距捷運南京復興站230公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_c_relevance,Data Specialist,台灣保樂力加股份有限公司,121.541777,25.051622,0,[]
1426,0,12850348,微生物(資深) 研究員_Microbiology Senior(Sr.) Scientist,微生物(資深) 研究員_Microbiology Senior(Sr.) <em class...,1,1,6001002025,新北市五股區,五工六路25號,Position Summary:\nPharmacology Discovery Serv...,...,[員工130人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_c_relevance,微生物(資深) 研究員_Microbiology Senior(Sr.) Scientist,汎球生物科技股份有限公司,121.448519,25.062759,0,"[生物學相關, 醫學技術及檢驗相關, 藥學相關]"
1427,2,12823438,資料工程師 Data Engineer,資料工程師 <em class='b-txt--highlight'>Data</em> E...,1,1,6001001005,台北市大安區,近忠孝復興捷運站,與PM/UI/UX、前後端、資料、演算法工程師一同合作，開發與維護產品。\n 2. 負...,...,[員工650人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_c_relevance,資料工程師 Data Engineer,東森新媒體控股股份有限公司,121.542599,25.041815,0,[]
1428,0,13029724,Data Engineer 數據工程師,<em class='b-txt--highlight'>Data</em> Enginee...,1,1,6001016015,高雄市路竹區,,do we manage the complexities of our manufact...,...,[員工260人],,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,jolist_c_relevance,Data Engineer 數據工程師,德商默克在台集團_美商慧盛先進科技有限公司台灣分公司,120.262811,22.849404,0,"[自然科學學科類, 數學及電算機科學學科類]"


### M3. Dump multiple variables to pickle

```python
import pickle
with open('104.pkl', 'wb') as fout:  # Python 3: open(..., 'wb')
    pickle.dump([all_data, df], fout)
```

In [26]:
# Uncomment the following code

import pickle
# with open('104.pkl', 'wb') as fout:  # Python 3: open(..., 'wb')
#     pickle.dump([all_data, df], fout)

#### Load multiple variables back to objects

```python
with open('104.pkl', "rb") as fin:  # Python 3: open(..., 'rb')
    test = pickle.load(fin)
    print(type(test[0]))
    print(type(test[1]))
```

In [91]:
# Uncomment the following code

# with open('104.pkl', "rb") as fin:  # Python 3: open(..., 'rb')
#     test = pickle.load(fin)
#     print(type(test[0]))
#     print(type(test[1]))

In [92]:
# Uncommment the following code

# test[0][0].keys()

## Using timestamp as file name

因為我們抓取資料的for-loop如果有幾千或幾萬個iterations，勢必很怕抓到一半斷掉或當機，卻又得重抓。所以最好的方式就是，每抓個幾百筆或幾千筆，就留一份檔案。但要如何區分這些檔案的先後順序？通常會用撈取的時間加上資料筆數的先後來作為檔名的一部分，這樣之後就可以觀看這些檔名來知道，究竟資料有沒有抓完，或者當在哪裡。

下面的程式碼將取得的現在時間加到檔名中

```python
with open('104_%s.pkl'%(now), 'wb') as fout:  # Python 3: open(..., 'wb')
    pickle.dump([all_data, df], fout)
```

In [29]:
from datetime import datetime

now = datetime.now().strftime("%Y%m%d%H%M%S")
print("Current Time =", now)

# with open('104_%s.pkl'%(now), 'wb') as fout:  # Python 3: open(..., 'wb')
#     pickle.dump([all_data, df], fout)

Current Time = 20221010120556


## Clean version. 104


In [93]:
headers = {'referer': 'https://www.104.com.tw/'}
search_str = '數據分析'

# Detecting totalPage
url_104 = 'https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=' + search_str + '&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=2&mode=s&jobsource=2018indexpoc'
raw = requests.get(url_104, headers=headers).json()
totalPage = raw['data']['totalPage']
print(totalPage)

# Getting data by loop
all_data = []
# for page in range(1, totalPage):
for page in range(1, 6):
    url = 'https://www.104.com.tw/jobs/search/list?ro=0&kwop=7&keyword=' + search_str + '&expansionType=area%2Cspec%2Ccom%2Cjob%2Cwf%2Cwktm&order=14&asc=0&page=' + str(page) + '&mode=s&jobsource=2018indexpoc'
    raw = requests.get(url, headers=headers).json()
    all_data.extend(raw['data']['list'])
    print(len(all_data), end = " ")
print()
df = pd.DataFrame(all_data)


# Saving and Backing up data
import pickle
from datetime import datetime

now = datetime.now().strftime("%Y%m%d%H%M%S")
print("Current Time =", now)


# with open('104_%s_%s.pkl'%(search_str, now), 'wb') as fout:  # Python 3: open(..., 'wb')
#     pickle.dump([all_data, df], fout)

150
30 50 70 90 110 
Current Time = 20221010134501
